In [1]:
!pip install openai pytube -q
!pip install git+https://github.com/openai/whisper.git -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.9 MB/s eta 0:00:00


In [2]:
import openai
import whisper
import pandas as pd
from pytube import YouTube
from getpass import getpass

COMPLETIONS_MODEL = "text-davinci-003"
EMBEDDINGS_MODEL = "text-embedding-ada-002"
openai.api_key = getpass("Enter your OpenAI API Key")

Enter your OpenAI API Key··········


In [3]:
model = whisper.load_model('base')

100%|███████████████████████████████████████| 139M/139M [00:03<00:00, 40.1MiB/s]


In [4]:
import threading
from pytube import YouTube
import os

video_urls = [
    "https://www.youtube.com/watch?v=0z49u_b6PmE",
    "https://www.youtube.com/watch?v=IDGnZ2AErBg",
    "https://www.youtube.com/watch?v=rBm9y4NuxlY",
    "https://www.youtube.com/watch?v=oazzjOjJXKQ",
    "https://www.youtube.com/watch?v=Gy0A-e1xNco"
]

def download_video(video_url):
    try:
        yt = YouTube(video_url)
        stream = yt.streams.filter(file_extension='mp4').first()
        if stream:
            download_path = os.path.join("videos", yt.title + ".mp4")
            stream.download(output_path="videos", filename=yt.title)
            print(f"Downloaded: {yt.title}.mp4")
        else:
            print(f"No mp4 stream available for {yt.title}")
    except Exception as e:
        print(f"Error downloading {video_url}: {str(e)}")

if not os.path.exists("videos"):
    os.makedirs("videos")

threads = []
for video_url in video_urls:
    thread = threading.Thread(target=download_video, args=(video_url,))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

print("All downloads completed.")

Downloaded: Why You Shouldn't Max Out Your 401k | Portfolio Rescue.mp4
Error downloading https://www.youtube.com/watch?v=rBm9y4NuxlY: [Errno 2] No such file or directory: '/content/videos/Is a 90/10 Portfolio Too Risky For Retirement? | Portfolio Rescue'
Downloaded: How Do I Teach My Kids About Money? | Portfolio Rescue.mp4
Downloaded: Should I Invest in Rental Properties? | Portfolio Rescue.mp4
Downloaded: Where Should I Save For My Down Payment? | Portfolio Rescue.mp4
All downloads completed.


In [10]:
output = model.transcribe("videos/financial_advisor.mp4")

In [11]:
output['text']

" Welcome back to our show portfolio rescue. This is a show where you, the viewers, hand up the topics through your questions. Our inbox is overflowing, dunking on it, and I don't have time to get to all of them. We're trying. If you have a question email us, ask the compound show at gmail.com or mind her, quick disclaimer, this show is for informational purposes only. It should not be relied upon for investment decisions. We're filming one day early today because tomorrow's Abbey Thanksgiving, dunking any big plans. I'm just getting together with some family. All right. I need some good food. All right. Think it's bar night of the year. Yes for you? No. No. It's amateur hour, right? Yeah. I'm actually running a five KMR one. Oh, okay. Nice. Turkey run. Is that what they call it? Yeah. Something like that. Yeah. All right. Remember, it's not just going to be me answering these questions. I'm going to answer some of them on my own, but I also have an expert waiting in the wings is going

In [22]:
import csv

# create a new pandas dataframe
df = pd.DataFrame(columns=['episode', 'url', 'start_timestamp', 'start', 'end', 'question', 'context'])

# index to keep track of the current row
i = 0

# Open the CSV file with the correct encoding (e.g., ISO-8859-1)
with open('financial_advisor.csv', encoding='ISO-8859-1') as csv_file:
    reader = csv.reader(csv_file)

    # Skip the header row in the CSV file
    next(reader)

    for row in reader:
        # Assign each column in the row to a variable and split questions on carriage return
        episode, url, questions = row
        question_list = questions.split("\n")

        # For each question in the list, extract the timestamp and convert it to seconds for YouTube
        for question in question_list:
            pieces = question.split('-')
            timestamp = pieces[0]
            minutes, seconds = timestamp.split(':')
            seconds = int(seconds) + (int(minutes.lstrip()) * 60)

            # Add a new row to the dataframe
            df.loc[i] = [episode, url, timestamp, seconds, seconds, " ".join(pieces[1:]), ""]

            try:
                df.loc[i-1]['end'] = df.loc[i]['start']
            except:
                print(f"skipping row {i} because there is no previous row")

            i += 1

            df['end'][df['end'] < df['start']] = 0
            df['end'][df['start'] == df['end']] = 0

df.to_csv("questions.csv")

skipping row 0 because there is no previous row


<ipython-input-22-758b48bdbc5e>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['end'][df['end'] < df['start']] = 0
<ipython-input-22-758b48bdbc5e>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['end'][df['start'] == df['end']] = 0
<ipython-input-22-758b48bdbc5e>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[i-1]['end'] = df.loc[i]['start']
<ipython-input-22-758b48bdbc5e>:38: SettingWithCopyWarning: 
A value is tr

In [13]:
import pandas as pd

df = pd.read_csv('questions.csv')
df

,Unnamed: 0,episode,url,start_timestamp,start,end,question,context
0,0,Episode 1,https://www.youtube.com/watch?v=0z49u_b6PmE,03:07,187,0,What's the best way to reallocate a portfolio...,NaN
1,1,Episode 1,https://www.youtube.com/watch?v=0z49u_b6PmE,06:55,415,0,What do I do once I cross the Roth IRA income...,NaN
2,2,Episode 1,https://www.youtube.com/watch?v=0z49u_b6PmE,11:09,669,0,Once I have maxed out my retiremenet accounts...,NaN
3,3,Episode 2,https://www.youtube.com/watch?v=IDGnZ2AErBg,01:04,64,0,Saving vs. Investing,NaN
4,4,Episode 2,https://www.youtube.com/watch?v=IDGnZ2AErBg,06:06,366,0,If Baby Boomers are shifting from net investo...,NaN
5,5,Episode 2,https://www.youtube.com/watch?v=IDGnZ2AErBg,09:26,566,0,What is your opinion on interest only mortgages?,NaN
6,6,Episode 2,https://www.youtube.com/watch?v=IDGnZ2AErBg,13:44,824,0,Advice on contributions to ESOP Employee Stoc...,NaN
7,7,Episode 3,https://www.youtube.com/watch?v=rBm9y4NuxlY,01:30,90,0,Is a 90/10 portfolio too risky in retirement?,NaN
8,8,Episode 3,https://www.youtube.com/watch?v=rBm9y4NuxlY,06:44,404,0,Considerations when buying a new construction...,NaN
9,9,Episode 3,https://www.youtube.com/watch?v=rBm9y4NuxlY,11:15,675,0,Should I consider adding municipal bonds to m...,NaN


In [23]:
# Let's just get the questions for a single episode and make this work before we download and transcribe all episodes in bulk
episode = df[df['episode'] == 'Episode 22'].copy()
episode

,episode,url,start_timestamp,start,end,question,context
83,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,01:58,118,0,How does the fed raising interest rates fight...,
84,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,03:30,210,0,What particular sectors/names do you find mos...,
85,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,05:16,316,0,What will bring more value in the long run fo...,
86,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,10:18,618,0,Would you recommend focusing on paying off my...,
87,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,11:57,717,0,Paying off a mortgage.,
88,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,13:25,805,0,What happens when the Fed raises rates?,
89,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,14:39,879,0,Im 44 and have a net worth of around $425K. ...,
90,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,17:19,1039,0,What are some good questions to ask when sele...,
91,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,21:15,1275,0,"At interest rates so low, should I instead t...",


In [24]:
def is_part_of_question(segment, start, end):
    if segment['start'] > start:
      if segment['end'] < end or end == 0:
          return True

    return False

def get_question_context(row):
     global output

     question_segments = list(filter(lambda segment: is_part_of_question(segment, row['start'], row['end']), output['segments']))
     # include question from timestamp in the context
     context = row['question']
     for segment in question_segments:
         context += segment['text']

     return context

episode['context'] = episode.apply(get_question_context, axis=1)

episode

,episode,url,start_timestamp,start,end,question,context
83,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,01:58,118,0,How does the fed raising interest rates fight...,How does the fed raising interest rates fight...
84,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,03:30,210,0,What particular sectors/names do you find mos...,What particular sectors/names do you find mos...
85,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,05:16,316,0,What will bring more value in the long run fo...,What will bring more value in the long run fo...
86,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,10:18,618,0,Would you recommend focusing on paying off my...,Would you recommend focusing on paying off my...
87,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,11:57,717,0,Paying off a mortgage.,Paying off a mortgage. retired in some way. S...
88,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,13:25,805,0,What happens when the Fed raises rates?,What happens when the Fed raises rates? And t...
89,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,14:39,879,0,Im 44 and have a net worth of around $425K. ...,Im 44 and have a net worth of around $425K. ...
90,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,17:19,1039,0,What are some good questions to ask when sele...,What are some good questions to ask when sele...
91,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,21:15,1275,0,"At interest rates so low, should I instead t...","At interest rates so low, should I instead t..."


In [25]:
from tenacity import retry, wait_fixed, stop_after_delay
from openai.embeddings_utils import get_embedding

# Define a retry decorator with a wait period and maximum retry time
@retry(wait=wait_fixed(20), stop=stop_after_delay(600))  # Retry every 20 seconds, stop after 10 minutes
def get_embedding_with_retry(text):
    return get_embedding(text, engine='text-embedding-ada-002')

# Use the retry decorator for embedding requests
episode['embedding'] = episode['context'].apply(lambda row: get_embedding_with_retry(row))
episode.to_csv('question_embeddings.csv')

In [27]:
from openai.embeddings_utils import cosine_similarity

question = "Should I buy a house with cash?"
question_vector = get_embedding(question, engine='text-embedding-ada-002')

episode["similarities"] = episode['embedding'].apply(lambda x: cosine_similarity(x, question_vector))
episode = episode.sort_values("similarities", ascending=False).head(4)

episode

,episode,url,start_timestamp,start,end,question,context,embedding,similarities
87,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,11:57,717,0,Paying off a mortgage.,Paying off a mortgage. retired in some way. S...,"[0.0007957892958074808, -0.030531644821166992,...",0.789473
91,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,21:15,1275,0,"At interest rates so low, should I instead t...","At interest rates so low, should I instead t...","[-0.014481776393949986, -0.035299330949783325,...",0.779268
83,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,01:58,118,0,How does the fed raising interest rates fight...,How does the fed raising interest rates fight...,"[-0.016737906262278557, -0.028229404240846634,...",0.777636
86,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,10:18,618,0,Would you recommend focusing on paying off my...,Would you recommend focusing on paying off my...,"[-0.00816159788519144, -0.025098249316215515, ...",0.776179


In [28]:
episode.to_csv("sorted.csv")

In [29]:
context = []
for i, row in episode.iterrows():
  context.append(row['context'])

context

[" Paying off a mortgage. retired in some way. So how do you think about this from a financial planning perspective, and also a portfolio management perspective from keeping this money safe? What do you think? Well, first of all, I just want to say he's done a really good job because for him, the value of money is the freedom it's going to afford him to do these projects that he wants to do. And he was very cognizant of that. And in doing that, he stockpiled a lot of cash, which is a very good thing. That being said, he does raise an interesting point, which is the whole inflation component. He's got a lot in cash. I'm hoping he didn't mean his 401k is in cash as well, because that should be invested differently, obviously. But the key here is he's still going to need quite a bit of cash because if he expects to be out of the full-time working world doing his project for one to three years, and he's spending about $30,000 a year, I would probably keep about $100,000 in cash. I was goin

In [30]:
text = (context)
text

[" Paying off a mortgage. retired in some way. So how do you think about this from a financial planning perspective, and also a portfolio management perspective from keeping this money safe? What do you think? Well, first of all, I just want to say he's done a really good job because for him, the value of money is the freedom it's going to afford him to do these projects that he wants to do. And he was very cognizant of that. And in doing that, he stockpiled a lot of cash, which is a very good thing. That being said, he does raise an interesting point, which is the whole inflation component. He's got a lot in cash. I'm hoping he didn't mean his 401k is in cash as well, because that should be invested differently, obviously. But the key here is he's still going to need quite a bit of cash because if he expects to be out of the full-time working world doing his project for one to three years, and he's spending about $30,000 a year, I would probably keep about $100,000 in cash. I was goin

In [31]:
!pip install gradio

In [32]:
import gradio as gr
import openai

In [33]:
context = "At interest rates so low, should I instead take a loan, finance the house and invest my capital in the stock market? So last but not least, I'm in my 30s and can now afford to buy a house full cash without taking any debt. However, with interest rates so low and they wrote this one a while back, but should I take a loan anyway and finance the house and invest my capital in the stock market? I think this is interesting that rates could change the story. Going from a 3% mortgage rate to a 5% that hurdle changes. So nine months ago, I would have said you'd be nuts to pay full cash now. Maybe it kind of makes sense, but first of all, who does this person for having that amount of money? I don't really live to be able to buy and cash, but that's great. But so Matt, you're thinking through this type of decision with a client. And this is the kind of thing where there really is no right or wrong answer, right? A lot of this is personality driven and depending on what the person wants to get"

def answer_question(question):
    prompt = f"""Answer the following question using only the context below. Answer in the style of Ben Carlson, a financial advisor, and podcaster.

Context:
{context}

Q: {question}
A:"""

    # Specify the GPT-3.5-turbo model when making the API request
    response = openai.Completion.create(
        engine="text-davinci-002",  # Specify the GPT-3.5-turbo engine
        prompt=prompt,
        temperature=0.7,
        max_tokens=500,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )["choices"][0]["text"].strip(" \n")

    return response

iface = gr.Interface(fn=answer_question, inputs="text", outputs="text", live=True, title="Financial Advisor Q&A")
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://175bce4070de5874b7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
